In [1]:
import numpy as np
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from data.base_data_module import BaseDataset

C:\Users\Filipe\.conda\envs\wav2kp\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [2]:
def scale_keypoints(keypoints, max_size=(256, 256)):
    # divide all x and y coordinates by the corresponding axis max
    # this is defined on the image processing stage
    keypoints[ :, 0] = keypoints[ :, 0]/max_size[0]
    keypoints[ :, 1] = keypoints[ :, 1]/max_size[1]
    
    return keypoints

In [3]:
# train_dataset = np.load("dataset/Train.npy", allow_pickle=True)
# test_dataset = np.load("dataset/Test.npy", allow_pickle=True)
# val_dataset = np.load("dataset/Val.npy", allow_pickle=True)

train_keypoints = np.load("dataset/Train_keypoints.npy",  allow_pickle=True)
test_keypoints = np.load("dataset/Test_keypoints.npy",  allow_pickle=True)
val_keypoints = np.load("dataset/Val_keypoints.npy",  allow_pickle=True)

train_mfccs = torch.load("dataset/Train_mfccs.pt" )
test_mfccs = torch.load("dataset/Test_mfccs.pt" )
val_mfccs = torch.load("dataset/Val_mfccs.pt" )

train_mfccs_not_transp = torch.load("dataset/Train_mfccs.pt" )
train_mfccs_not_transp = torch.transpose(train_mfccs_not_transp, 2, 1)

train_set  = BaseDataset(data=train_mfccs, targets=train_keypoints, target_transform=scale_keypoints)

print(train_mfccs_not_transp.shape)
train_set  = BaseDataset(data=train_mfccs_not_transp, targets=train_keypoints, target_transform=scale_keypoints)


train_loader = DataLoader(train_set)
# test_loader = DataLoader(test_dataset)
# val_loader = DataLoader(val_dataset)

torch.Size([80, 40, 5300])


In [4]:


train_keypoints = np.load("dataset/Train_keypoints.npy",  allow_pickle=True)


train_mfccs = torch.load("dataset/Train_mfccs.pt" )


train_mfccs_not_transp = torch.load("dataset/Train_mfccs.pt" )
train_mfccs_not_transp = torch.transpose(train_mfccs_not_transp, 2, 1)

train_set  = BaseDataset(data=train_mfccs, targets=train_keypoints, target_transform=scale_keypoints)
train_set_nt  = BaseDataset(data=train_mfccs_not_transp, targets=train_keypoints, target_transform=scale_keypoints)

train_loader = DataLoader(train_set)
train_loader_nt = DataLoader(train_set_nt)


In [5]:
lstm_input_size = train_set[0][0].shape[0]

In [6]:
train_set.__len__()

80

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
# Here we define our model as a class
class LSTM(pl.LightningModule):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2, dnn_shape=530000):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.dnn_shape = dnn_shape

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True)

        # Define the output layer
        self.linear = nn.Linear(self.dnn_shape, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        #print(f"input.view(len(input) {input.view(len(input), self.batch_size, -1).shape}")
        # lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        lstm_out, self.hidden = self.lstm(input)
        #print(lstm_out.shape)
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        #print(f"y_pred.shape in net: {y_pred.shape}")
        return y_pred.view(-1)


    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        # scheduler = StepLR(optimizer, step_size=800, gamma=0.5)

        return optimizer

    def training_step(self, batch, batch_idx):
        X_batch, y_batch = batch
        #print(f" X_batch:{X_batch.shape}, y_batch: {y_batch.shape}")
        y_pred = self.forward(X_batch)
        #print(f"y_pred.shape: {y_pred.shape}, y_batch.shape(): {y_batch.shape}")
        loss_fn = torch.nn.MSELoss()
        loss = loss_fn(y_pred, y_batch.float())
        # criterion = nn.BCEWithLogitsLoss()
        # loss = criterion(y_pred, y_batch.unsqueeze(1))
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss}
        


    
    def validation_step(self, batch):
        X_batch, y_batch = batch
        y_pred = self.forward(X_batch)
        
        return {'loss': loss}

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        # scheduler = StepLR(optimizer, step_size=800, gamma=0.5)

        return optimizer

model = LSTM(40, 100, batch_size=1, output_dim=136, num_layers=2)

In [8]:

if True:
    logger = pl.loggers.TensorBoardLogger("training/logs")


    trainer = pl.Trainer(logger=logger, weights_save_path="training/logs", gpus=1 )

    trainer.fit(model, train_loader)
    # trainer.test(model, datamodule=data)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
C:\Users\Filipe\.conda\envs\wav2kp\lib\site-packages\pytorch_lightning\utilities\distributed.py:68: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params
----------------------------------
0 | lstm   | LSTM   | 137 K 
1 | linear | Linear | 72.1 M
----------------------------------
72.2 M    Trainable params
0         Non-trainable params
72.2 M    Total params
288.871   Total estimated model params size (MB)
Epoch 0:   0%|          | 0/80 [00:00<?, ?it/s] C:\Users\Filipe\.conda\envs\wav2kp\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([1, 722, 136])) that is different to the input size (torch.Size([136])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_l

In [9]:
aaaaa

train_keypoints = np.load("dataset/Train_keypoints.npy",  allow_pickle=True)


train_mfccs = torch.load("dataset/Train_mfccs.pt" )


train_mfccs_not_transp = torch.load("dataset/Train_mfccs.pt" )
train_mfccs_not_transp = torch.transpose(train_mfccs_not_transp, 2, 1)

train_set  = BaseDataset(data=train_mfccs, targets=train_keypoints, target_transform=scale_keypoints)
train_set_nt  = BaseDataset(data=train_mfccs_not_transp, targets=train_keypoints, target_transform=scale_keypoints)

train_loader = DataLoader(train_set)
train_loader_nt = DataLoader(train_set_nt)


NameError: name 'aaaaa' is not defined

In [63]:
it = iter(train_loader)
first_mfcc, first_kp = next(it)
print( first_mfcc.shape, first_mfcc.size(-1))
print( first_kp.shape)

torch.Size([1, 5300, 40]) 40
torch.Size([1, 722, 136])


In [66]:
first_kp[0][0]

tensor([3.9339e-06, 7.7486e-06, 6.6000e+01, 1.4500e+02, 6.9000e+01, 1.5900e+02,
        7.1000e+01, 1.7200e+02, 7.5000e+01, 1.8300e+02, 8.5000e+01, 1.9500e+02,
        9.2000e+01, 2.0000e+02, 1.0600e+02, 2.0600e+02, 1.2300e+02, 2.1200e+02,
        1.4200e+02, 2.1000e+02, 1.5300e+02, 2.0400e+02, 1.6200e+02, 2.0000e+02,
        1.7200e+02, 1.9100e+02, 1.7900e+02, 1.8000e+02, 1.8300e+02, 1.6600e+02,
        1.8900e+02, 1.5300e+02, 1.9300e+02, 1.3800e+02, 8.1000e+01, 1.2300e+02,
        9.0000e+01, 1.2100e+02, 1.0000e+02, 1.2100e+02, 1.0900e+02, 1.2300e+02,
        1.1700e+02, 1.2500e+02, 1.4700e+02, 1.2600e+02, 1.5500e+02, 1.2500e+02,
        1.6400e+02, 1.2500e+02, 1.7400e+02, 1.2500e+02, 1.8100e+02, 1.2800e+02,
        1.3000e+02, 1.4000e+02, 1.3000e+02, 1.4900e+02, 1.3000e+02, 1.5700e+02,
        1.2800e+02, 1.6400e+02, 1.1700e+02, 1.6600e+02, 1.2300e+02, 1.6800e+02,
        1.2800e+02, 1.7000e+02, 1.3400e+02, 1.7000e+02, 1.3800e+02, 1.6800e+02,
        9.2000e+01, 1.3600e+02, 9.8000e+

In [41]:
print(torch.reshape(first_kp[0,0], (1, 136)).squeeze()[0], torch.reshape(first_kp[0,0], (1, 136)).squeeze()[-1])
print(first_kp[0,0, 0],first_kp[0,0, -1] )

tensor(0.2578, dtype=torch.float64) tensor(181., dtype=torch.float64)
tensor(0.2578, dtype=torch.float64) tensor(181., dtype=torch.float64)


In [42]:
lstm = nn.LSTM(40, 100, 2, batch_first=True)


In [43]:
print( first_mfcc.shape, first_mfcc.size(-1))


torch.Size([1, 5300, 40]) 40


In [44]:
lstm_out, hl = lstm(first_mfcc)

In [45]:
lstm_out[-1].view(1, -1).shape

torch.Size([1, 530000])

In [46]:
lin = nn.Linear(530000, 136)


In [48]:
lin_out = lin(lstm_out[-1].view(1, -1))
lin_out.shape

torch.Size([1, 136])

In [ ]:
loss_fn = torch.nn.MSELoss()
        loss = loss_fn(y_pred, y_batch)

In [13]:
train_mfccs[0].size(-1)

40

In [14]:
if True:
    logger = pl.loggers.TensorBoardLogger("training/logs")


    trainer = pl.Trainer(logger=logger, weights_save_path="training/logs" )

    trainer.fit(model, train_loader)
    # trainer.test(model, datamodule=data)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores

  | Name   | Type   | Params
----------------------------------
0 | lstm   | LSTM   | 137 K 
1 | linear | Linear | 6.9 K 
----------------------------------
144 K     Trainable params
0         Non-trainable params
144 K     Total params
0.578     Total estimated model params size (MB)
Epoch 0:   0%|          | 0/80 [00:00<?, ?it/s] 40
40
40
40
Epoch 0:   0%|          | 0/80 [00:00<?, ?it/s]


RuntimeError: input.size(-1) must be equal to input_size. Expected 40, got 21200

In [15]:
if True:
    logger = pl.loggers.TensorBoardLogger("training/logs")


    trainer = pl.Trainer(logger=logger, weights_save_path="training/logs" )

    trainer.fit(model, train_loader_nt)
    # trainer.test(model, datamodule=data)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores

  | Name   | Type   | Params
----------------------------------
0 | lstm   | LSTM   | 137 K 
1 | linear | Linear | 6.9 K 
----------------------------------
144 K     Trainable params
0         Non-trainable params
144 K     Total params
0.578     Total estimated model params size (MB)
Epoch 0:   0%|          | 0/80 [00:00<?, ?it/s] 5300
5300
5300
5300
Epoch 0:   0%|          | 0/80 [00:00<?, ?it/s]


RuntimeError: input.size(-1) must be equal to input_size. Expected 40, got 21200

In [17]:
for element in train_set :
    print (element[0].shape)
    
for element in train_mfccs_not_transp:
    print (element.shape)
    

40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Size([5300, 40])
40
torch.Siz

In [18]:
torch.transpose(element, 1, 0).size(-1)

40

In [9]:
loss_fn = torch.nn.MSELoss(size_average=False)

optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)

#####################
# Train model
#####################

hist = np.zeros(num_epochs)

for t in range(num_epochs):
    # Clear stored gradient
    model.zero_grad()
    
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    model.hidden = model.init_hidden()
    
    # Forward pass
    y_pred = model(X_train)

    loss = loss_fn(y_pred, y_train)
    if t % 100 == 0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

C:\Users\Filipe\.conda\envs\wav2kp\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


NameError: name 'model' is not defined

In [7]:
# TODO: make these args from the argparser
trainer = pl.Trainer(gpus=1)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [6]:
 mfcc, kp = train_set[1]

In [7]:
mfcc.shape

torch.Size([5300, 40])

In [ ]:
class bc_dsft_half(pl.LightningModule):
    def __init__(self, learning_rate):
        super(bc_dsft_half, self).__init__()
        self.learning_rate = learning_rate
        self.layer_1 = nn.Linear(158, 158)
        self.layer_2 = nn.Linear(158, 128)
        self.layer_3 = nn.Linear(128, 64)
        self.layer_4 = nn.Linear(64, 32)
        self.layer_5 = nn.Linear(32, 16)
        self.layer_6 = nn.Linear(16, 8)
        self.layer_7 = nn.Linear(8, 4)
        self.layer_out = nn.Linear(4, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))

        x = self.relu(self.layer_2(x))
        x = self.dropout(x)

        x = self.relu(self.layer_3(x))
        x = self.dropout(x)

        x = self.relu(self.layer_4(x))
        x = self.dropout(x)

        x = self.relu(self.layer_5(x))
        x = self.dropout(x)

        x = self.relu(self.layer_6(x))
        x = self.dropout(x)

        x = self.relu(self.layer_7(x))
        x = self.dropout(x)

        x = self.layer_out(x)

        return x

    def train_dataloader(self):
        # train_dataset_path = "datasets/train_prepared_dataset_no_encode_dsft.parquet"
        # batch_size = 32*32*256
        # batch_size = 32*32*256
        # train_loader, test_loader, y_test, scaler = get_train_data(train_dataset_path,
        #                                                            batch_size,
        #                                                            RS=7,
                                                                #    scaler=StandardScaler())
        return train_loader

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        # scheduler = StepLR(optimizer, step_size=800, gamma=0.5)

        return optimizer

    def training_step(self, batch, batch_idx):
        X_batch, y_batch = batch
        y_pred = self.forward(X_batch)
        criterion = nn.BCEWithLogitsLoss()
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        tensorboard_logs = {'train_loss': loss}

        return {'loss': loss,  'log': tensorboard_logs}

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        criterion = nn.BCEWithLogitsLoss()
        loss = criterion(y_hat, y.unsqueeze(1))

        # f1 = f1_score(y_hat, y.unsqueeze(1))
        tensorboard_logs = {'valid_loss': loss}

        return {'val_loss': loss, 'log': tensorboard_logs}

    def validation_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}

        return {'val_loss': avg_loss,  'log': tensorboard_logs}

    def test_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        criterion = nn.BCEWithLogitsLoss()
        loss = criterion(y_hat, y.unsqueeze(1))


        tensorboard_logs = {'test_loss': loss }
        # , 'test_f1': classification_report(y, y_hat)}

        return {'test_loss': loss, 'log': tensorboard_logs}

    def test_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        logs = {'test_loss': avg_loss}
        #print(classification_report(y.unsqueeze(1).cpu().numpy(), y_hat.cpu().numpy().round()))

        return {'test_loss': avg_loss, 'log': logs, 'progress_bar': logs}

    def val_dataloader(self):
        # OPTIONAL
        return valid_loader

    def test_dataloader(self):
        # OPTIONAL
        return test_loader


if __name__ == '__main__':
    train_loader, valid_loader, test_loader = generate_dataloaders("datasets/train_prepared_dataset_no_encode_dsft.parquet",
                                                                   "datasets/test_prepared_dataset_no_encode_dsft.parquet",
                                                                   RS=RS, batch_size=256*256*256)

    net = bc_dsft_half(learning_rate=0.001)


    early_stopping = EarlyStopping('val_loss', verbose=True, patience=100)
    checkpoint_callback = ModelCheckpoint('checkpoints/', monitor='val_loss', save_top_k=1 )
    
    # logger = TensorBoardLogger(
    #     save_dir=os.getcwd(),
    #     version=1,
    #     name='lightning_logs_bc_dsft_half'
    # )

    trainer = pl.Trainer(gpus=1, 
                        early_stop_callback=early_stopping, 
                        checkpoint_callback=checkpoint_callback,
                        auto_lr_find=False,
                        benchmark=True,
                        #check_val_every_n_epoch=10, 
                        max_epochs=4000, 
                        # logger=logger, 
                        profiler=True)

    # lr_finder = trainer.lr_find(net)

    # print(lr_finder.results)
    # new_lr = lr_finder.suggestion()
    # print(f'lr_finder.suggestion() {lr_finder.suggestion()}')
    # net.hparams.learning_rate = new_lr

    trainer.fit(net)

    trainer.test()
